#  MCP client running in jupyter notebook -- using SSE model

Experiment on building MCP client using SSE model running in jupyter notebook. 

It connects to postgres MCP server (SSE), get and print available tools, then run a sample query


# MCP postgresql server (SSE) is rebuilt referenced to official server list on modelcontextprotocol.io

@modelcontextprotocol/server-postgres

rebuilt server is provided as: 

./server/postgres/postgres.py


# references

https://github.com/sidharthrajaram/mcp-sse


In [ ]:
# references:  https://github.com/sidharthrajaram/mcp-sse
# ./server/postgres/postgres.py -- MCP server was rebuilt from its nodejs version to python

import asyncio
import json
import os
import sys
from typing import Optional

from mcp import ClientSession
from mcp.client.sse import sse_client

from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

class MCPClient:
    def __init__(self):
        # Initialize session and context variables
        self.session: Optional[ClientSession] = None
        self._streams_context = None
        self._session_context = None

    async def initialize(self, server_url: str):
        """
        Connect to an MCP server running with SSE transport and initialize the session.
        
        Parameters:
            server_url (str): URL of the SSE MCP server (e.g. "http://localhost:8080/sse")
        """
        # Enter the SSE client context and create the MCP session
        self._streams_context = sse_client(url=server_url)
        streams = await self._streams_context.__aenter__()

        self._session_context = ClientSession(*streams)
        self.session = await self._session_context.__aenter__()

        # Initialize the session handshake with the MCP server
        await self.session.initialize()
        print("Initialized SSE client.")

    async def get_tools(self):
        """
        Retrieve the list of tools from the MCP server.
        
        Returns:
            The list of tool objects provided by the server.
        """
        response = await self.session.list_tools()
        return response.tools
    
    async def call_tool(self, tool_name: str, parameters: dict): 
        """
        Call a specific tool on the MCP server with given parameters.
        
        Parameters:
            tool_name (str): The name of the tool to call.
            parameters (dict): The parameters to pass to the tool.
        
        Returns:
            The response from the MCP server after calling the tool.
        """
        response = await self.session.call_tool(tool_name, parameters)
        return response
    

    async def get_tools_openai_format(self):
        """
        Convert the retrieved MCP tools into a simplified OpenAI tool format.
        
        Returns:
            A list of dictionaries each containing:
              - "name": tool name.
              - "description": a description of the tool.
              - "parameters": a dictionary representing tool parameters (from inputSchema).
        """
        tools = await self.get_tools()
        openai_tools = []
        for tool in tools:
            openai_tool = {
                "type": "function",
                "function": {
                    "name": getattr(tool, "name", "unknown_tool"),
                    "description": getattr(tool, "description", "No description provided"),
                    "parameters": getattr(tool, "inputSchema", {})
                }
            }
            openai_tools.append(openai_tool)
        return openai_tools

    async def cleanup(self):
        """Clean up the MCP session and underlying stream contexts."""
        if self._session_context:
            await self._session_context.__aexit__(None, None, None)
        if self._streams_context:
            await self._streams_context.__aexit__(None, None, None)


server_url = "http://localhost:8081/sse"

async def main():

    client = MCPClient()
    try:
        # Initialize the MCP client
        await client.initialize(server_url)
        
        # Retrieve tools and convert to OpenAI format
        openai_tools = await client.get_tools_openai_format()
        print("Available tools in OpenAI format:")
        print(json.dumps(openai_tools, indent=2))

        # test of call_tool
        res = await client.call_tool("query", {"sql": "SELECT * FROM transactions limit 10"})

        print(res)

        if hasattr(res, 'content') and res.content:
            text_response = res.content[0].text
            try:
                parsed_result = json.loads(text_response)
                print("\n🔹 Query Result:")
                print(json.dumps(parsed_result, indent=2))
            except json.JSONDecodeError:
                print("\n⚠️ Response is not valid JSON:", text_response)
        
    except Exception as e:
        print("Error during initialization or tool retrieval:", e)
    finally:
        await client.cleanup()

await main()
